In [10]:
import wrds
import pandas as pd

conn = wrds.Connection()


Loading library list...
Done


In [14]:
df = pd.read_csv("/Users/chasewilliamson/Library/CloudStorage/Dropbox/carbon_policy_reallocation/data/raw/ORBIS/orbis_nace.csv")[["bvdid","CTRYISO", "NACEPCOD2"]]

df.dropna(subset=["NACEPCOD2"], inplace=True)
# Convert NACEPCOD2 to string to handle numeric codes
df["NACEPCOD2"] = df["NACEPCOD2"].astype(str)

# Conditional slicing for 2-character codes
df.loc[df["NACEPCOD2"].str.len() == 5, "NACEPCOD2"] = "0" + df.loc[df["NACEPCOD2"].str.len() == 5, "NACEPCOD2"]

# Create NACE2 column based on conditions
df["NACE2"] = df["NACEPCOD2"].str[:2]  # Default case for all lengths


# De-duplicate to obtain unique 2 digit NACE x country pairs
unique_combinations = df.drop_duplicates(subset=['CTRYISO', 'NACE2'])[['CTRYISO', 'NACE2']]
unique_combinations = unique_combinations[unique_combinations['CTRYISO'] != 'US'] # US shouldn't be in here, but there were 2 ETS firms with this ISO code
# Group by country and collect NACE2 codes
grouped_combinations = unique_combinations.groupby('CTRYISO')['NACE2'].apply(list).to_dict()

In [16]:
# Initialize an empty list to store the data chunks
data_chunks = []

# Loop through each unique country and pull the data
for ctryiso, nace2_list in grouped_combinations.items():
    # Construct the WHERE clause for NACE2 codes
    nace2_conditions = " OR ".join([f"LEFT(nacepcod2, 2) = '{nace2}'" for nace2 in nace2_list])
    
    query = f"""
        SELECT bvdid, 
            ctryiso, 
            LEFT(nacepcod2, 2) AS NACE2
        FROM bvd.ob_w_company_profiles_lms
        WHERE ctryiso = '{ctryiso}' 
            AND ({nace2_conditions})
    """
    
    # Fetch data for the current country
    df_chunk = pd.DataFrame(conn.raw_sql(query))
    
    # Append the data chunk to the list
    data_chunks.append(df_chunk)

    # Print status message
    print("Completed fetching data for", ctryiso)

# Concatenate all data chunks into a single DataFrame (if needed)
final_df = pd.concat(data_chunks, ignore_index=True)

# Display the final DataFrame
display(final_df)


Completed fetching data for AT
Completed fetching data for BE
Completed fetching data for BG
Completed fetching data for CH
Completed fetching data for CY
Completed fetching data for CZ
Completed fetching data for DE
Completed fetching data for DK
Completed fetching data for EE
Completed fetching data for ES
Completed fetching data for FI
Completed fetching data for FR
Completed fetching data for GB
Completed fetching data for GR
Completed fetching data for HR
Completed fetching data for HU
Completed fetching data for IE
Completed fetching data for IS
Completed fetching data for IT
Completed fetching data for LT
Completed fetching data for LU
Completed fetching data for LV
Completed fetching data for MT
Completed fetching data for NL
Completed fetching data for NO
Completed fetching data for PL
Completed fetching data for PT
Completed fetching data for RO
Completed fetching data for SE
Completed fetching data for SI
Completed fetching data for SK


,bvdid,ctryiso,nace2
0,AT*110306067026,AT,62
1,AT*470450116,AT,66
2,AT*550539882,AT,47
3,AT*552006406,AT,64
4,AT*642617693,AT,85
...,...,...,...
48903641,SK55143555,SK,68
48903642,SKFEB19019,SK,64
48903643,SKFEB42683,SK,64
48903644,SKFEB44804,SK,64


In [21]:
final_df[["bvdid"]].to_csv("/Users/chasewilliamson/Library/CloudStorage/Dropbox/carbon_policy_reallocation/data/raw/ORBIS/orbis_bvdids_full.csv", index=False)

In [22]:
final_df.to_csv("/Users/chasewilliamson/Library/CloudStorage/Dropbox/carbon_policy_reallocation/data/raw/ORBIS/orbis_nace2_full.csv", index=False)

In [52]:
bvdid_list1 = final_df.iloc[:1000000,0]

In [42]:
bvdid_list1

0        AT*110306067026
1           AT*470450116
2           AT*550539882
3           AT*552006406
4           AT*642617693
              ...       
99995       AT9030271209
99996       AT9030271211
99997       AT9030271222
99998       AT9030271227
99999       AT9030267511
Name: bvdid, Length: 100000, dtype: object

In [56]:
# Step 1: Construct the SQL query with IN clause
id_list = final_df["bvdid"].tolist()
id_str = ','.join([f"'{id}'" for id in id_list])  # Surround each ID with single quotes

query = f"""
    SELECT 
        bvdid, 
        ctryiso, 
        closdate, 
        exchrate, 
        av, 
        turn, 
        staf, 
        tfas, 
        conscode
    FROM bvd.ob_w_ind_g_fins_cfl_usd_int_lms
    WHERE bvdid IN ({id_str})"""
    
full_orbis = pd.DataFrame(conn.raw_sql(query) )


In [55]:
final_df[["bvdid"]]["bvdid"]

0           AT*110306067026
1              AT*470450116
2              AT*550539882
3              AT*552006406
4              AT*642617693
                 ...       
48903641         SK55143555
48903642         SKFEB19019
48903643         SKFEB42683
48903644         SKFEB44804
48903645         SKFEB49407
Name: bvdid, Length: 48903646, dtype: object